In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from faker import Faker
from models import Professor, Disciplina, Turma, Aluno, Nota, Base

## Relacionamentos:
- Turma é a combinação ANO|Semestre|Disciplina
- Um professor pode estar associado a várias turmas, então é um relacionamento de um para muitos entre Professores e Turmas.
- Cada professor só está associado a uma Disciplina (id_disciplina = id_professor)
- Uma disciplina pode ter várias turmas em diferentes anos e semestres, então é um relacionamento de um para muitos entre Disciplinas e Turmas.
- Um aluno pode estar matriculado em várias turmas, então é um relacionamento de um para muitos entre Alunos e Turmas.
- Cada registro de Nota está associado a um único aluno e a uma única turma, portanto, é um relacionamento de um para um entre Alunos, Turmas e Notas.

## Criação do database

In [1]:

# Criando a conexão com o banco de dados
engine = create_engine('sqlite:///desempenho_academico.db')

# Criando as tabelas no banco de dados
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

# Instanciando Faker para geração de dados fictícios
fake = Faker()

# Populando tabela de professores
for _ in range(5):
    professor = Professor(
        nome=fake.name(),
        email=fake.email(),
        senha=fake.password(),
        papel='Professor'
    )
    session.add(professor)

# Populando tabela de disciplinas
disciplinas = [
    {"nome": "Matemática", "codigo": "MAT101"},
    {"nome": "Física", "codigo": "FIS101"},
    {"nome": "Química", "codigo": "QUI101"},
    {"nome": "Biologia", "codigo": "BIO101"},
    {"nome": "História", "codigo": "HIS101"}
]

for disciplina_data in disciplinas:
    disciplina = Disciplina(**disciplina_data)
    session.add(disciplina)

# Criando turmas para cada disciplina
for disciplina in session.query(Disciplina):
    for ano in range(2022, 2025):
        for semestre in ['A', 'B']:
            turma = Turma(
                ano=ano,
                semestre=semestre,
                disciplina_id=disciplina.id
            )
            session.add(turma)

# Populando tabela de alunos
for _ in range(10):
    aluno = Aluno(
        nome=fake.name(),
        matricula=fake.unique.random_number(digits=8)
    )
    session.add(aluno)

# Populando tabela de notas
for aluno in session.query(Aluno):
    for turma in session.query(Turma):
        nota = Nota(
            aluno_id=aluno.id,
            turma_id=turma.id,
            notas=fake.random_int(min=0, max=10)
        )
        session.add(nota)

# Commit para salvar as alterações no banco de dados
session.commit()

# Fechando a sessão
session.close()
